# Minimax (and Alpha–Beta) - expectimax — *Othello*



## Rules summary (Othello)

- You must place a disc of your color on an **empty** square such that it **flanks** one or more contiguous enemy discs in at least one direction (horizontal, vertical, diagonal).  
- All flanked enemy discs in those directions are **flipped** to your color.
- If a player has **no legal moves**, they must **pass**; if **both** players must pass, the game ends.
- Winner: player with **more discs** at the end (ties are possible).


In [15]:
from __future__ import annotations

from dataclasses import dataclass
from typing import List, Tuple, Optional, Iterable, Callable, Dict
import math
import random
import copy
import math, random
from typing import Optional, Tuple, Callable, List

In [16]:
N = 8
BLACK, WHITE, EMPTY = 1, -1, 0
PLAYERS = {BLACK: "BLACK", WHITE: "WHITE"}

DIRS = [(-1, -1), (-1, 0), (-1, 1),
        ( 0, -1),          ( 0, 1),
        ( 1, -1), ( 1, 0), ( 1, 1)]

def opponent(p: int) -> int:
    return -p

def on_board(r: int, c: int) -> bool:
    return 0 <= r < N and 0 <= c < N

def initial_board() -> List[List[int]]:
    b = [[EMPTY for _ in range(N)] for _ in range(N)]
    b[3][3] = WHITE; b[3][4] = BLACK
    b[4][3] = BLACK; b[4][4] = WHITE
    return b

def copy_board(b):
    return [row[:] for row in b]


In [17]:
def _flips_in_dir(board: List[List[int]], r: int, c: int, dr: int, dc: int, player: int) -> List[Tuple[int,int]]:
    """Return the list of opponent positions that would be flipped in one direction, else []"""
    flips = []
    r += dr; c += dc
    if not on_board(r,c) or board[r][c] != opponent(player):
        return []
    while on_board(r,c) and board[r][c] == opponent(player):
        flips.append((r,c))
        r += dr; c += dc
    if not on_board(r,c) or board[r][c] != player:
        return []
    return flips

def legal_moves(board: List[List[int]], player: int) -> List[Optional[Tuple[int,int]]]:
    """Return a list of legal moves (r,c); if none exist, the only legal move is [None] (pass)."""
    moves = []
    for r in range(N):
        for c in range(N):
            if board[r][c] != EMPTY:
                continue
            flips_any = []
            for dr,dc in DIRS:
                flips_any += _flips_in_dir(board, r, c, dr, dc, player)
            if flips_any:
                moves.append((r,c))
    if not moves:
        return [None]
    return moves

def apply_move(board: List[List[int]], move: Optional[Tuple[int,int]], player: int) -> List[List[int]]:
    """Return a NEW board after applying move (or pass=None)."""
    if move is None:
        return copy_board(board)
    r, c = move
    assert board[r][c] == EMPTY, "illegal: target not empty"
    flips_all = []
    for dr,dc in DIRS:
        flips_all += _flips_in_dir(board, r, c, dr, dc, player)
    assert flips_all, "illegal: no flips created"
    nb = copy_board(board)
    nb[r][c] = player
    for fr,fc in flips_all:
        nb[fr][fc] = player
    return nb

def count_discs(board: List[List[int]]) -> Tuple[int,int,int]:
    b = sum(cell == BLACK for row in board for cell in row)
    w = sum(cell == WHITE for row in board for cell in row)
    e = N*N - b - w
    return b, w, e


In [18]:
def _flips_in_dir(board: List[List[int]], r: int, c: int, dr: int, dc: int, player: int) -> List[Tuple[int,int]]:
    """Return the list of opponent positions that would be flipped in one direction, else []"""
    flips = []
    r += dr; c += dc
    if not on_board(r,c) or board[r][c] != opponent(player):
        return []
    while on_board(r,c) and board[r][c] == opponent(player):
        flips.append((r,c))
        r += dr; c += dc
    if not on_board(r,c) or board[r][c] != player:
        return []
    return flips

def legal_moves(board: List[List[int]], player: int) -> List[Optional[Tuple[int,int]]]:
    """Return a list of legal moves (r,c); if none exist, the only legal move is [None] (pass)."""
    moves = []
    for r in range(N):
        for c in range(N):
            if board[r][c] != EMPTY:
                continue
            flips_any = []
            for dr,dc in DIRS:
                flips_any += _flips_in_dir(board, r, c, dr, dc, player)
            if flips_any:
                moves.append((r,c))
    if not moves:
        return [None]
    return moves

def apply_move(board: List[List[int]], move: Optional[Tuple[int,int]], player: int) -> List[List[int]]:
    """Return a NEW board after applying move (or pass=None)."""
    if move is None:
        return copy_board(board)
    r, c = move
    assert board[r][c] == EMPTY, "illegal: target not empty"
    flips_all = []
    for dr,dc in DIRS:
        flips_all += _flips_in_dir(board, r, c, dr, dc, player)
    assert flips_all, "illegal: no flips created"
    nb = copy_board(board)
    nb[r][c] = player
    for fr,fc in flips_all:
        nb[fr][fc] = player
    return nb

def count_discs(board: List[List[int]]) -> Tuple[int,int,int]:
    b = sum(cell == BLACK for row in board for cell in row)
    w = sum(cell == WHITE for row in board for cell in row)
    e = N*N - b - w
    return b, w, e


In [19]:
def print_board(board: List[List[int]]):
    header = "    " + " ".join(str(c) for c in range(N))
    print(header)
    print("   " + "--"*N)
    for r in range(N):
        row = []
        for c in range(N):
            v = board[r][c]
            row.append("●" if v == BLACK else "○" if v == WHITE else ".")
        print(f"{r:>2} | " + " ".join(row))
    b,w,e = count_discs(board)
    print(f"    (●=BLACK {b}, ○=WHITE {w}, empty {e})")

def pretty_move(m):
    return "pass" if m is None else f"({m[0]},{m[1]})"


In [20]:
class Agent:
    name = "Agent"
    def action(self, state: OthelloState) -> Optional[Tuple[int,int]]:
        raise NotImplementedError

class RandomAgent(Agent):
    name = "Random"
    def action(self, state: OthelloState):
        b = state.to_board()
        moves = legal_moves(b, state.player)
        return random.choice(moves)

class GreedyFlipAgent(Agent):
    """Pick the move that flips the most discs immediately; pass if only pass."""
    def __init__(self):
        self.name = "GreedyFlips"
    def action(self, state: OthelloState):
        b = state.to_board()
        moves = legal_moves(b, state.player)
        if moves == [None]:
            return None
        best = None
        best_flips = -1
        for m in moves:
            r,c = m
            flips = 0
            for dr,dc in DIRS:
                flips += len(_flips_in_dir(b, r, c, dr, dc, state.player))
            if flips > best_flips:
                best_flips, best = flips, m
        return best

def play_game(black_agent: Agent, white_agent: Agent, seed: Optional[int] = None, verbose: bool = False):
    rng = random.Random(seed)
    random.seed(seed)
    board = initial_board()
    player = BLACK
    state = OthelloState.from_board(board, player)
    history = []
    if verbose:
        print_board(board)
    while not is_terminal(state):
        agent = black_agent if state.player == BLACK else white_agent
        move = agent.action(state)
        moves = legal_moves(state.to_board(), state.player)
        assert move in moves, f"{agent.name} chose illegal move {move}; legal are {moves}"
        if move is None:
            history.append((state.player, None))
            state = OthelloState.from_board(state.to_board(), opponent(state.player))
            if verbose:
                print(f"{PLAYERS[opponent(state.player)]} passes.")
            continue
        new_board = apply_move(state.to_board(), move, state.player)
        history.append((state.player, move))
        state = OthelloState.from_board(new_board, opponent(state.player))
        if verbose:
            print(f"{PLAYERS[opponent(state.player)]} played {pretty_move(move)}")
            print_board(new_board)

    b,w,e = count_discs(state.to_board())
    result = 1 if b > w else -1 if w > b else 0
    if verbose:
        print("Final:")
        print_board(state.to_board())
        if result == 0:
            print("Draw.")
        else:
            print(f"Winner: {PLAYERS[result]}")
    return result, history  


## Evaluation function 

In [21]:
CORNER_CELLS = [(0,0), (0,7), (7,0), (7,7)]

def dynamic_weight(total_discs: int) -> List[List[float]]:
    """
    Returns a dynamically adjusted weight matrix based on the current game phase.
    """
    if total_discs < 20:
        return [
            [ 4, 3,  2,  2,  2,  2,  3,  4],
            [ 3, 2,  1,  1,  1,  1,  2,  3],
            [ 2, 1,  1,  0,  0,  1,  1,  2],
            [ 2, 1,  0,  1,  1,  0,  1,  2],
            [ 2, 1,  0,  1,  1,  0,  1,  2],
            [ 2, 1,  1,  0,  0,  1,  1,  2],
            [ 3, 2,  1,  1,  1,  1,  2,  3],
            [ 4, 3,  2,  2,  2,  2,  3,  4]
        ]
    elif total_discs < 50:
        return [
            [ 3, 2,  1,  1,  1,  1,  2,  3],
            [ 2, 2,  1,  1,  1,  1,  2,  2],
            [ 1, 1,  1,  1,  1,  1,  1,  1],
            [ 1, 1,  1,  1,  1,  1,  1,  1],
            [ 1, 1,  1,  1,  1,  1,  1,  1],
            [ 1, 1,  1,  1,  1,  1,  1,  1],
            [ 2, 2,  1,  1,  1,  1,  2,  2],
            [ 3, 2,  1,  1,  1,  1,  2,  3]
        ]
    else:
        return [
            [ 1, 1,  1,  1,  1,  1,  1,  1],
            [ 1, 1,  1,  1,  1,  1,  1,  1],
            [ 1, 1,  1,  1,  1,  1,  1,  1],
            [ 1, 1,  1,  1,  1,  1,  1,  1],
            [ 1, 1,  1,  1,  1,  1,  1,  1],
            [ 1, 1,  1,  1,  1,  1,  1,  1],
            [ 1, 1,  1,  1,  1,  1,  1,  1],
            [ 1, 1,  1,  1,  1,  1,  1,  1]
        ]
        
def evaluate(board: List[List[int]], perspective: int) -> float:
    """
    Returns a scalar value representing the evaluation of the board from the
    perspective of the current player.
    """
    b, w, e = count_discs(board)
    
    discs_score = b / (b + w) if perspective == BLACK else w / (b + w)
    
    my_moves = len(legal_moves(board, perspective))
    opp_moves = len(legal_moves(board, opponent(perspective)))
    move_score = my_moves / (my_moves + opp_moves) if (my_moves + opp_moves) > 0 else 0

    
    corner_score = 0
    for (r, c) in CORNER_CELLS:
        if board[r][c] == perspective:
            corner_score += 0.25
    
    weight_matrix = dynamic_weight(b + w)
    edge_score = 0
    for r in range(8):
        for c in range(8):
            if board[r][c] == perspective:
                edge_score += weight_matrix[r][c]
    
    final_score = 0.5 * discs_score + 0.2 * move_score + 0.15 * corner_score + 0.15 * edge_score
    return final_score

## Implement **Minimax**

Implement a `MinimaxAgent` that:
- Searches to a fixed **depth**.
- Uses `evaluate(board, perspective)` at depth cutoffs (non-terminal leaves).
- Treats the current player as the **maximizer** and the opponent as **minimizer**.
- Returns the **best legal move** for `state.player`.

Note: use self.nodes and count the number of nodes visited

In [22]:
class MinimaxAgent(Agent):
    def __init__(self, depth: int = 4, eval_fn: Callable[[List[List[int]], int], float] = evaluate):
        self.depth = depth
        self.eval_fn = eval_fn
        self.name = f"Minimax(d={depth})"
        self.nodes = 0  

    def action(self, state: OthelloState) -> Optional[Tuple[int, int]]:
        perspective = state.player
        board = state.to_board()
        moves = legal_moves(board, state.player)

        if moves == [None]:
            return None

        best_move = None
        best_val = -math.inf

        for move in moves:
            next_board = apply_move(board, move, state.player)
            next_state = OthelloState(next_board, opponent(state.player))
            val = self._min_value(next_state, self.depth - 1, perspective)
            if val > best_val:
                best_val = val
                best_move = move
                
        print(f"{self.name} explored {self.nodes} nodes.")
        return best_move


    def _max_value(self, state: OthelloState, depth: int, perspective: int) -> float:
        self.nodes += 1
        if is_terminal(state):
            return utility(state, perspective)

        if depth <= 0:
            self.nodes += 1
            return self.eval_fn(state.to_board(), perspective)

        board = state.to_board()
        moves = legal_moves(board, state.player)

        if moves == [None]:
            next_board = apply_move(board, None, state.player)
            next_state = OthelloState(next_board, opponent(state.player))
            return self._min_value(next_state, depth - 1, perspective)

        value = -math.inf

        for move in moves:
            next_board = apply_move(board, move, state.player)
            next_state = OthelloState(next_board, opponent(state.player))
            val = self._min_value(next_state, depth - 1, perspective)
            value = max(value, val)
        return value


    def _min_value(self, state: OthelloState, depth: int, perspective: int) -> float:
        self.nodes += 1
        if is_terminal(state):
            return utility(state, perspective)

        if depth <= 0:
            return self.eval_fn(state.to_board(), perspective)

        board = state.to_board()
        moves = legal_moves(board, state.player)

        if moves == [None]:
            next_board = apply_move(board, None, state.player)
            next_state = OthelloState(next_board, opponent(state.player))
            return self._max_value(next_state, depth - 1, perspective)

        value = math.inf

        for move in moves:
            next_board = apply_move(board, move, state.player)
            next_state = OthelloState(next_board, opponent(state.player))
            value_temp = self._max_value(next_state, depth - 1, perspective)
            value = min(value, value_temp)

        return value



## **Alpha–Beta pruning**

Implement `AlphaBetaAgent` with the same interface but pruned search:
- Maintain `alpha` (best value for max so far) and `beta` (best for min).
- Prune when `value >= beta` at max nodes, or `value <= alpha` at min nodes.
- Keep the same evaluation function and depth semantics.

Note: use self.nodes and count the number of nodes visited

In [23]:
class AlphaBetaAgent(Agent):
    def __init__(self, depth: int = 5, eval_fn: Callable[[List[List[int]], int], float] = evaluate):
        self.depth = depth
        self.eval_fn = eval_fn
        self.name = f"AlphaBeta(d={depth})"
        self.nodes = 0

    def action(self, state: OthelloState) -> Optional[Tuple[int, int]]:
        perspective = state.player
        b = state.to_board()
        moves = legal_moves(b, state.player)

        if moves == [None]:
            return None

        def order_key(m):
            next_board = apply_move(b, m, state.player)
            next_state = OthelloState.from_board(next_board, opponent(state.player))
            return self.eval_fn(next_state.to_board(), perspective)

        moves.sort(key=order_key, reverse=True)

        alpha, beta = -math.inf, math.inf
        best_move, best_val = None, -math.inf

        for move in moves:
            next_board = apply_move(b, move, state.player)
            next_state = OthelloState.from_board(next_board, opponent(state.player))
            val = self._min_value(next_state, self.depth - 1, perspective, alpha, beta)
            if val > best_val:
                best_val = val
                best_move = move
            alpha = max(alpha, best_val)
            if beta <= alpha:
                break  
        
        print(f"{self.name} explored {self.nodes} nodes.")
        return best_move


    def _max_value(self, state: OthelloState, depth: int, perspective: int, alpha: float, beta: float) -> float:
        self.nodes += 1
        if is_terminal(state):
            return utility(state, perspective)

        if depth <= 0:
            return self.eval_fn(state.to_board(), perspective)

        b = state.to_board()
        moves = legal_moves(b, state.player)

        if moves == [None]:
            next_board = apply_move(b, None, state.player)
            next_state = OthelloState.from_board(next_board, opponent(state.player))
            return self._min_value(next_state, depth - 1, perspective, alpha, beta)

        def order_key(m):
            next_board = apply_move(b, m, state.player)
            next_state = OthelloState.from_board(next_board, opponent(state.player))
            return self.eval_fn(next_state.to_board(), perspective)

        moves.sort(key=order_key, reverse=True)

        value = -math.inf
        for move in moves:
            next_board = apply_move(b, move, state.player)
            next_state = OthelloState.from_board(next_board, opponent(state.player))
            val = self._min_value(next_state, depth - 1, perspective, alpha, beta)
            value = max(value, val)
            alpha = max(alpha, value)
            if beta <= alpha:
                break  
        return value


    def _min_value(self, state: OthelloState, depth: int, perspective: int, alpha: float, beta: float) -> float:
        self.nodes += 1
        if is_terminal(state):
            return utility(state, perspective)

        if depth <= 0:
            return self.eval_fn(state.to_board(), perspective)

        b = state.to_board()
        moves = legal_moves(b, state.player)

        if moves == [None]:
            next_board = apply_move(b, None, state.player)
            next_state = OthelloState.from_board(next_board, opponent(state.player))
            return self._max_value(next_state, depth - 1, perspective, alpha, beta)

        def order_key(m):
            next_board = apply_move(b, m, state.player)
            next_state = OthelloState.from_board(next_board, opponent(state.player))
            return self.eval_fn(next_state.to_board(), perspective)

        moves.sort(key=order_key, reverse=True)

        value = math.inf
        for move in moves:
            next_board = apply_move(b, move, state.player)
            next_state = OthelloState.from_board(next_board, opponent(state.player))
            val = self._max_value(next_state, depth - 1, perspective, alpha, beta)
            value = min(value, val)
            beta = min(beta, value)
            if beta <= alpha:
                break  
        return value


## Expectimax with Imperfect Opponent (Risk-aware)

In this part you’ll implement **Expectimax** where the **opponent is modeled as a chance node**:

- With probability **(1 − ε)** the opponent plays the **best** move for them (i.e., the move that **minimizes our value**).
- With probability **ε** the opponent plays **a suboptimal move**, chosen **uniformly among the remaining legal moves**.

This allows our agent to sometimes choose a move that is *risky* under strict minimax, but has **higher expected value** if the opponent occasionally blunders.

Note: use self.nodes and count the number of nodes visited

In [24]:
import math
from typing import Optional, Tuple, Callable, List

class ExpectimaxRiskyAgent(Agent):
    """
    Expectimax with an *imperfect opponent*:
      - With prob (1 - eps), opponent plays the minimizing move (best for them).
      - With prob eps, opponent plays uniformly among the *other* legal moves.

    depth:        search depth in plies
    eval_fn:      evaluation(board, perspective) -> float   (use your own evaluate)
    blunder_rate: epsilon in [0,1]
    """
    def __init__(self,
                 depth: int = 4,
                 eval_fn: Callable[[List[List[int]], int], float] = evaluate,
                 blunder_rate: float = 0.20):
        self.depth = depth
        self.eval_fn = eval_fn
        self.eps = max(0.0, min(1.0, blunder_rate))
        self.name = f"ExpectimaxRisky(d={depth}, eps={self.eps:.2f})"
        self.nodes = 0

    def action(self, state: OthelloState) -> Optional[Tuple[int,int]]:
        """
        TODO: Return the legal move that maximizes our *expected* value under the imperfect-opponent model.
        - If no legal move (only [None]), return None (pass).
        - For each legal move m:
            compute expected value after m by calling _opponent_expectation on the child state.
        - Pick the argmax by value.
        """
        board = state.to_board()
        moves = legal_moves(board, state.player)
        if moves == [None]:
            return None

        perspective = state.player
        best_move, best_val = None, -math.inf
        for move in moves:
            next_board = apply_move(board, move, state.player)
            next_state = OthelloState.from_board(next_board, opponent(state.player))
            val = self._opponent_expectation(next_state, self.depth - 1, perspective)
            if val > best_val:
                best_val = val
                best_move = move

        print(f"{self.name} explored {self.nodes} nodes.")
        return best_move

    def _max_value(self, state: OthelloState, depth: int, perspective: int) -> float:
        self.nodes += 1
        if is_terminal(state):
            return utility(state, perspective)

        if depth <= 0:
            return self.eval_fn(state.to_board(), perspective)

        b = state.to_board()
        moves = legal_moves(b, state.player)

        if moves == [None]:
            next_board = apply_move(b, None, state.player)
            next_state = OthelloState.from_board(next_board, opponent(state.player))
            return self._opponent_expectation(next_state, depth - 1, perspective)

        value = -math.inf
        for move in moves:
            next_board = apply_move(b, move, state.player)
            next_state = OthelloState.from_board(next_board, opponent(state.player))
            val = self._opponent_expectation(next_state, depth - 1, perspective)
            value = max(value, val)
        return value

    def _opponent_expectation(self, state: OthelloState, depth: int, perspective: int) -> float:
        self.nodes += 1
        if is_terminal(state):
            return utility(state, perspective)

        if depth <= 0:
            return self.eval_fn(state.to_board(), perspective)

        b = state.to_board()
        moves = legal_moves(b, state.player)

        if moves == [None]:
            next_board = apply_move(b, None, state.player)
            next_state = OthelloState.from_board(next_board, opponent(state.player))
            return self._max_value(next_state, depth - 1, perspective)

        vals = []
        for m in moves:
            next_board = apply_move(b, m, state.player)
            next_state = OthelloState.from_board(next_board, opponent(state.player))
            v = self._max_value(next_state, depth - 1, perspective)
            vals.append((m, v))

        vals.sort(key=lambda x: x[1])
        best_move, best_val = vals[0]
        others = vals[1:]

        if len(vals) == 1 or self.eps <= 1e-12:
            return best_val

        eps = self.eps
        avg_others = sum(v for (_, v) in others) / len(others)
        exp_val = (1 - eps) * best_val + eps * avg_others
        return exp_val


## engine checks

Run these to verify rules implementation (not search).


In [25]:
b = initial_board()
print_board(b)

lm_black = legal_moves(b, BLACK)
lm_white = legal_moves(b, WHITE)
assert len([m for m in lm_black if m is not None]) == 4
assert len([m for m in lm_white if m is not None]) == 4

assert (2,3) in lm_black
b2 = apply_move(b, (2,3), BLACK)
print_board(b2)

print("✓ Basic engine checks passed.")


    0 1 2 3 4 5 6 7
   ----------------
 0 | . . . . . . . .
 1 | . . . . . . . .
 2 | . . . . . . . .
 3 | . . . ○ ● . . .
 4 | . . . ● ○ . . .
 5 | . . . . . . . .
 6 | . . . . . . . .
 7 | . . . . . . . .
    (●=BLACK 2, ○=WHITE 2, empty 60)
    0 1 2 3 4 5 6 7
   ----------------
 0 | . . . . . . . .
 1 | . . . . . . . .
 2 | . . . ● . . . .
 3 | . . . ● ● . . .
 4 | . . . ● ○ . . .
 5 | . . . . . . . .
 6 | . . . . . . . .
 7 | . . . . . . . .
    (●=BLACK 4, ○=WHITE 1, empty 59)
✓ Basic engine checks passed.


## Try it out

Play a few games between baseline agents and your search agents.

In [26]:
minimax_agent = MinimaxAgent(depth=3)
ab_agent = AlphaBetaAgent(depth=3)

res, _ = play_game(minimax_agent, ab_agent, seed=10, verbose=True)
print(f"Minimax vs AlphaBeta (Depth 3): {res} | nodes: {minimax_agent.nodes} / {ab_agent.nodes}")

expectimax_10 = ExpectimaxRiskyAgent(depth=3, blunder_rate=0.1)
expectimax_30 = ExpectimaxRiskyAgent(depth=3, blunder_rate=0.3)
expectimax_50 = ExpectimaxRiskyAgent(depth=3, blunder_rate=0.5)

res_10, _ = play_game(expectimax_10, minimax_agent, seed=11, verbose=True)
res_30, _ = play_game(expectimax_30, minimax_agent, seed=12, verbose=True)
res_50, _ = play_game(expectimax_50, minimax_agent, seed=13, verbose=True)

print(f"Expectimax (10% error) vs Minimax: {res_10} | nodes: {expectimax_10.nodes}")
print(f"Expectimax (30% error) vs Minimax: {res_30} | nodes: {expectimax_30.nodes}")
print(f"Expectimax (50% error) vs Minimax: {res_50} | nodes: {expectimax_50.nodes}")


    0 1 2 3 4 5 6 7
   ----------------
 0 | . . . . . . . .
 1 | . . . . . . . .
 2 | . . . . . . . .
 3 | . . . ○ ● . . .
 4 | . . . ● ○ . . .
 5 | . . . . . . . .
 6 | . . . . . . . .
 7 | . . . . . . . .
    (●=BLACK 2, ○=WHITE 2, empty 60)
Minimax(d=3) explored 72 nodes.
BLACK played (2,3)
    0 1 2 3 4 5 6 7
   ----------------
 0 | . . . . . . . .
 1 | . . . . . . . .
 2 | . . . ● . . . .
 3 | . . . ● ● . . .
 4 | . . . ● ○ . . .
 5 | . . . . . . . .
 6 | . . . . . . . .
 7 | . . . . . . . .
    (●=BLACK 4, ○=WHITE 1, empty 59)
AlphaBeta(d=3) explored 51 nodes.
WHITE played (4,2)
    0 1 2 3 4 5 6 7
   ----------------
 0 | . . . . . . . .
 1 | . . . . . . . .
 2 | . . . ● . . . .
 3 | . . . ● ● . . .
 4 | . . ○ ○ ○ . . .
 5 | . . . . . . . .
 6 | . . . . . . . .
 7 | . . . . . . . .
    (●=BLACK 3, ○=WHITE 3, empty 58)
Minimax(d=3) explored 220 nodes.
BLACK played (5,5)
    0 1 2 3 4 5 6 7
   ----------------
 0 | . . . . . . . .
 1 | . . . . . . . .
 2 | . . . ● . . . .
 3 | .

## Human vs Agent
play with your ai

You need to install and enable ipywidgets for the GUI to work.

**On Mac/Linux**: Usually works after `pip install ipywidgets`

**On Windows**: You may need to install and enable the extension:
```python
! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension --sys-prefix
! jupyter nbextension enable --py --sys-prefix widgetsnbextension
```

Run the cell below to check your setup and install if needed.

In [27]:
import sys
import subprocess

def install_and_setup_widgets():
    """Install and enable ipywidgets for Jupyter"""
    print("Checking ipywidgets installation...")
    
    try:
        import ipywidgets as widgets
        print("✓ ipywidgets is already installed")
    except ImportError:
        print("Installing ipywidgets...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "ipywidgets", "-q"])
        print("✓ ipywidgets installed")
    
    import platform
    if platform.system() == "Windows":
        print("\nEnabling Jupyter widget extensions for Windows...")
        try:
            subprocess.check_call([sys.executable, "-m", "jupyter", "nbextension", "enable", "--py", "widgetsnbextension", "--sys-prefix"], 
                                 stderr=subprocess.DEVNULL)
            print("✓ Extension enabled")
        except:
            print("Note: Extension enable commands may need manual execution")
            print("Run: jupyter nbextension enable --py widgetsnbextension --sys-prefix")
    
    print("\n✓ Setup complete! You may need to refresh your browser if widgets don't appear.")
    print("On Windows: If the GUI still doesn't show, try restarting Jupyter notebook.")

install_and_setup_widgets()


Checking ipywidgets installation...
✓ ipywidgets is already installed

Enabling Jupyter widget extensions for Windows...
Note: Extension enable commands may need manual execution
Run: jupyter nbextension enable --py widgetsnbextension --sys-prefix

✓ Setup complete! You may need to refresh your browser if widgets don't appear.
On Windows: If the GUI still doesn't show, try restarting Jupyter notebook.


In [28]:
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML
    WIDGETS_AVAILABLE = True
except ImportError as e:
    print("ERROR: ipywidgets is not installed or not working.")
    print("Please run the setup cell above to install it.")
    print(f"Error details: {e}")
    print("\nIf you're on Windows, you may need to run these commands in terminal:")
    print("  pip install ipywidgets jupyter nbextension enable --py widgetsnbextension --sys-prefix")
    WIDGETS_AVAILABLE = False

if not WIDGETS_AVAILABLE:
    raise ImportError("Cannot start GUI without ipywidgets")

class OthelloGUI:
    def __init__(self):
        self.state = None
        self.agent = None
        self.human_color = None
        self.game_active = False
        self.output = widgets.Output()
        self.last_ai_move = None
        
        self.cell_size = 60  
        self.setup_menu()
    
    def setup_menu(self):
        """Setup the agent selection menu"""
        clear_output(wait=True)
        title = widgets.HTML("<h2>🎮 Othello: Human vs AI</h2>")
        
        agent_dropdown = widgets.Dropdown(
            options=[
                ('Random', 'random'),
                ('Greedy Flips', 'greedy'),
                ('Minimax (depth=3)', 'minimax_3'),
                ('Minimax (depth=4)', 'minimax_4'),
                ('Alpha-Beta (depth=3)', 'alphabeta_3'),
                ('Alpha-Beta (depth=4)', 'alphabeta_4'),
                ('Alpha-Beta (depth=5)', 'alphabeta_5'),
                ('Expectimax (depth=3, ε=0.2)', 'expectimax_3'),
                ('Expectimax (depth=4, ε=0.2)', 'expectimax_4'),
            ],
            value='alphabeta_4',
            description='AI Agent:',
            style={'description_width': '100px'}
        )
        
        color_dropdown = widgets.Dropdown(
            options=[('Black (●) - Go First', BLACK), ('White (○) - Go Second', WHITE)],
            value=BLACK,
            description='Your Color:',
            style={'description_width': '100px'}
        )
        
        start_button = widgets.Button(
            description='Start Game',
            button_style='success',
            icon='play'
        )
        
        def on_start_clicked(b):
            agent_type = agent_dropdown.value
            if agent_type == 'random':
                self.agent = RandomAgent()
            elif agent_type == 'greedy':
                self.agent = GreedyFlipAgent()
            elif agent_type == 'minimax_3':
                self.agent = MinimaxAgent(depth=3)
            elif agent_type == 'minimax_4':
                self.agent = MinimaxAgent(depth=4)
            elif agent_type == 'alphabeta_3':
                self.agent = AlphaBetaAgent(depth=3)
            elif agent_type == 'alphabeta_4':
                self.agent = AlphaBetaAgent(depth=4)
            elif agent_type == 'alphabeta_5':
                self.agent = AlphaBetaAgent(depth=5)
            elif agent_type == 'expectimax_3':
                self.agent = ExpectimaxRiskyAgent(depth=3, blunder_rate=0.2)
            elif agent_type == 'expectimax_4':
                self.agent = ExpectimaxRiskyAgent(depth=4, blunder_rate=0.2)
            
            self.human_color = color_dropdown.value
            self.start_game()
        
        start_button.on_click(on_start_clicked)
        
        menu_box = widgets.VBox([
            title,
            agent_dropdown,
            color_dropdown,
            start_button
        ])
        
        display(menu_box)
    
    def start_game(self):
        """Start a new game"""
        clear_output(wait=True)
        
        board = initial_board()
        self.state = OthelloState.from_board(board, BLACK)
        self.game_active = True
        self.last_ai_move = None
        
        self.create_game_ui()
        
        if self.state.player != self.human_color:
            self.make_ai_move()
    
    def create_game_ui(self):
        """Create the game board UI"""
        self.info_label = widgets.HTML()
        
        self.buttons = []
        rows = []
        
        display(HTML("""
        <style>
        .othello-cell button {
            border-radius: 50% !important;
            font-size: 24px !important;
            font-weight: bold !important;
            color: white !important;
            padding: 0 !important;
            transition: all 0.2s ease;
            box-shadow: inset 0 2px 4px rgba(0,0,0,0.2);
        }
        .othello-cell button:hover:not(:disabled) {
            transform: scale(1.15);
            box-shadow: 0 6px 12px rgba(0,0,0,0.4), inset 0 2px 4px rgba(0,0,0,0.2);
            cursor: pointer;
        }
        </style>
        """))
        
        for r in range(N):
            row = []
            for c in range(N):
                btn = widgets.Button(
                    description='',
                    layout=widgets.Layout(
                        width=f'{self.cell_size}px', 
                        height=f'{self.cell_size}px',
                        margin='2px'
                    )
                )
                btn.add_class('othello-cell')
                btn.on_click(lambda b, row=r, col=c: self.on_cell_click(row, col))
                row.append(btn)
            rows.append(widgets.HBox(row, layout=widgets.Layout(margin='0')))
            self.buttons.append(row)
        
        board_box = widgets.VBox(
            rows, 
            layout=widgets.Layout(
                margin='10px 0px',
                padding='15px',
                background='#1b5e20',
                border_radius='10px'
            )
        )
        
        self.pass_button = widgets.Button(
            description='Pass',
            button_style='warning',
            icon='forward',
            layout=widgets.Layout(width='150px')
        )
        self.pass_button.on_click(lambda b: self.on_pass())
        
        new_game_button = widgets.Button(
            description='New Game',
            button_style='info',
            icon='refresh',
            layout=widgets.Layout(width='150px')
        )
        new_game_button.on_click(lambda b: self.setup_menu())
        
        self.status_output = widgets.Output()
        
        controls = widgets.HBox([self.pass_button, new_game_button])
        game_ui = widgets.VBox([
            self.info_label,
            board_box,
            controls,
            self.status_output
        ])
        
        display(game_ui)
        self.update_display()
    
    def update_display(self):
        """Update the board display"""
        board = self.state.to_board()
        legal = legal_moves(board, self.state.player) if self.game_active else []
        
        for r in range(N):
            for c in range(N):
                btn = self.buttons[r][c]
                cell = board[r][c]
                
                btn.description = ''
                btn.disabled = True
                
                is_last_move = self.last_ai_move and (r, c) == self.last_ai_move
                
                if cell == BLACK:
                    if is_last_move:
                        btn.style.button_color = '#000000'
                        btn.layout.border = '3px solid #FFA500'
                    else:
                        btn.style.button_color = '#000000'
                        btn.layout.border = '2px solid #888'
                    btn.disabled = True
                    
                elif cell == WHITE:
                    if is_last_move:
                        btn.style.button_color = '#F5F5DC'
                        btn.layout.border = '3px solid #FFA500'
                    else:
                        btn.style.button_color = '#F5F5DC'
                        btn.layout.border = '2px solid #888'
                    btn.disabled = True
                    
                elif self.game_active and self.state.player == self.human_color and (r, c) in legal:
                    btn.description = '+'
                    btn.style.button_color = '#28a745'
                    btn.layout.border = '2px solid #1e7e34'
                    btn.disabled = False
                    
                else:
                    btn.style.button_color = '#2d6a4f'
                    btn.layout.border = '2px solid #1b4332'
                    btn.disabled = True
        
        b, w, e = count_discs(board)
        info_html = f"""
        <div style='font-size: 18px; margin-bottom: 10px; padding: 10px; background: #f0f0f0; border-radius: 5px;'>
            <b>⚫ Black:</b> {b} | <b>⚪ White:</b> {w} | <b>Empty:</b> {e}<br>
        """
        
        if self.game_active:
            current_player = "⚫ Black" if self.state.player == BLACK else "⚪ White"
            if self.state.player == self.human_color:
                info_html += f"<b style='color: green;'>🎯 Your turn ({current_player})</b><br>"
                if self.last_ai_move:
                    info_html += f"<small style='color: #ff8800;'>🔶 Orange = AI's last move at ({self.last_ai_move[0]}, {self.last_ai_move[1]})</small>"
            else:
                info_html += f"<b style='color: blue;'>🤖 AI's turn ({current_player})</b>"
        else:
            if b > w:
                winner = "⚫ Black"
            elif w > b:
                winner = "⚪ White"
            else:
                winner = "🤝 Draw"
            info_html += f"<b style='color: red;'>🏁 Game Over! Winner: {winner}</b>"
        
        info_html += "</div>"
        self.info_label.value = info_html
        
        if self.game_active and self.state.player == self.human_color and legal == [None]:
            self.pass_button.disabled = False
        else:
            self.pass_button.disabled = True
    
    def on_cell_click(self, row, col):
        """Handle cell click"""
        if not self.game_active or self.state.player != self.human_color:
            return
        
        board = self.state.to_board()
        legal = legal_moves(board, self.state.player)
        
        if (row, col) not in legal:
            return
        
        new_board = apply_move(board, (row, col), self.state.player)
        self.state = OthelloState.from_board(new_board, opponent(self.state.player))
        
        self.last_ai_move = None
        
        with self.status_output:
            clear_output(wait=True)
            print(f"You played ({row}, {col})")
        
        self.update_display()
        
        if is_terminal(self.state):
            self.end_game()
            return
        
        if self.state.player != self.human_color:
            self.make_ai_move()
    
    def on_pass(self):
        """Handle pass button"""
        if not self.game_active or self.state.player != self.human_color:
            return
        
        board = self.state.to_board()
        legal = legal_moves(board, self.state.player)
        
        if legal != [None]:
            return
        
        self.state = OthelloState.from_board(board, opponent(self.state.player))
        
        self.last_ai_move = None
        
        with self.status_output:
            clear_output(wait=True)
            print("You passed.")
        
        self.update_display()
        
        if is_terminal(self.state):
            self.end_game()
            return
        
        if self.state.player != self.human_color:
            self.make_ai_move()
    
    def make_ai_move(self):
        """Make AI move"""
        import time
        
        time.sleep(0.3)
        
        board = self.state.to_board()
        legal = legal_moves(board, self.state.player)
        
        if legal == [None]:
            self.state = OthelloState.from_board(board, opponent(self.state.player))
            self.last_ai_move = None
            with self.status_output:
                clear_output(wait=True)
                print(f"AI ({self.agent.name}) passed.")
            self.update_display()
            
            if is_terminal(self.state):
                self.end_game()
            return
        
        move = self.agent.action(self.state)
        
        if move is not None:
            new_board = apply_move(board, move, self.state.player)
            self.state = OthelloState.from_board(new_board, opponent(self.state.player))
            
            self.last_ai_move = move
            
            with self.status_output:
                clear_output(wait=True)
                print(f"AI ({self.agent.name}) played ({move[0]}, {move[1]})")
        
        self.update_display()
        
        if is_terminal(self.state):
            self.end_game()
    
    def end_game(self):
        """End the game"""
        self.game_active = False
        self.update_display()
        
        b, w, e = count_discs(self.state.to_board())
        
        with self.status_output:
            clear_output(wait=True)
            if b > w:
                winner = "Black"
                if self.human_color == BLACK:
                    print(f"🎉 You won! Black: {b}, White: {w}")
                else:
                    print(f"😞 AI won! Black: {b}, White: {w}")
            elif w > b:
                winner = "White"
                if self.human_color == WHITE:
                    print(f"🎉 You won! White: {w}, Black: {b}")
                else:
                    print(f"😞 AI won! White: {w}, Black: {b}")
            else:
                print(f"🤝 Draw! Black: {b}, White: {w}")

gui = OthelloGUI()
